<a href="https://colab.research.google.com/github/john-jiang-ca/MachineLearning/blob/master/ch13_exe10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
from tensorflow import keras
from keras.layers import Dense, Flatten
from pathlib import Path
from matplotlib import pyplot as plt
import pandas as pd
import math


from tensorflow.train import Example, Features, Feature, Int64List, BytesList

# Download data and split the paths into training, validate and test set

In [2]:
DOWNLOAD_ROOT = "http://ai.stanford.edu/~amaas/data/sentiment/"
FILENAME = "aclImdb_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
path = Path(filepath).parent / "aclImdb"
path


84131840/84125825 [==============================] - 7s 0us/step


PosixPath('/root/.keras/datasets/aclImdb')

In [3]:
def get_paths(path: Path):
  return [str(p) for p in path.glob("*.txt")]

train_neg = get_paths(path / "train" / "neg")
train_pos = get_paths(path / "train" / "pos")
test_neg = get_paths(path / "test" / "neg")
test_pos = get_paths(path / "test" / "pos")
val_neg = test_neg[:7500]
val_pos = test_pos[:7500]
test_neg = test_neg[7500:]
test_pos = test_pos[7500:]

# Create dataset that can be executed out core

In [4]:
def build_dataset_without_in_mem_fit(positive_paths, negative_paths):
  pos_set = tf.data.TextLineDataset(positive_paths, num_parallel_reads=10).map(lambda review: (review, 1))
  neg_set = tf.data.TextLineDataset(negative_paths, num_parallel_reads=10).map(lambda review: (review, 0))
  return tf.data.Dataset.concatenate(pos_set, neg_set)

In [5]:
training_set = build_dataset_without_in_mem_fit(train_pos, train_neg).shuffle(len(train_pos)*2).prefetch(1)
validation_set = build_dataset_without_in_mem_fit(val_pos, val_neg).shuffle(len(train_pos)*2).prefetch(1)
test_set = build_dataset_without_in_mem_fit(test_pos, test_neg).shuffle(len(train_pos)*2).prefetch(1)

## Explore what training set

*   Take some data from the training set
*   Create a pandas Datafraem using the data taken above
*  Display the data



In [6]:
demo_data = training_set.take(5)
data = []
for item in demo_data:
  review, label = item
  data.append({
  "review": review.numpy(),
  "label": label.numpy()
  })
df = pd.DataFrame(data)
pd.set_option('display.max_colwidth', -1)
df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  # Remove the CWD from sys.path while we load stuff.


review  label
0  b"Why? Why did they make this movie? If Timothy Olyphant wasn't shirtless in it several times, there would be ABSOLUTELY no reason to watch this movie, ever. Um...Plot? Nope. Well-defined characters? nope. The only time I laughed was when my boyfriend made fun of the whole she-bang. P.S. Andy Dick? Nope."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Create TFRecord dataset

In [7]:
def create_example(item):
  review, label = item
  return Example(
      features=Features(
          feature={
              "review": Feature(bytes_list = BytesList(value=[review.numpy()])),
              "label": Feature(int64_list = Int64List(value=[label]))
          }
      )
  )

In [8]:
def write_record(name, dataset, n_shards):
  paths = ["/content/drive/MyDrive/data/{}.tfrecord-{:02d}-of-{:02d}".format(name, index, n_shards) for index in range(n_shards)]
  writers = [tf.io.TFRecordWriter(path) for path in paths]
  print(paths)
  index = 0
  for item in dataset:
    writer = writers[index % n_shards]
    writer.write(create_example(item).SerializeToString())
    index = index+1

In [9]:
write_record("train-set", training_set, 5)

['/content/drive/MyDrive/data/train-set.tfrecord-00-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-01-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-02-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-03-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-04-of-05']


In [10]:
write_record("test-set", test_set, 2)
write_record("validate-set", validation_set, 2)

['/content/drive/MyDrive/data/test-set.tfrecord-00-of-02', '/content/drive/MyDrive/data/test-set.tfrecord-01-of-02']
['/content/drive/MyDrive/data/validate-set.tfrecord-00-of-02', '/content/drive/MyDrive/data/validate-set.tfrecord-01-of-02']


Define constants for paths

In [11]:
TRAIN_PATHS = ['/content/drive/MyDrive/data/train-set.tfrecord-00-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-01-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-02-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-03-of-05', '/content/drive/MyDrive/data/train-set.tfrecord-04-of-05']
TEST_PATHS = ['/content/drive/MyDrive/data/test-set.tfrecord-00-of-02', '/content/drive/MyDrive/data/test-set.tfrecord-01-of-02']
VALIDATE_PATHS = ['/content/drive/MyDrive/data/validate-set.tfrecord-00-of-02', '/content/drive/MyDrive/data/validate-set.tfrecord-01-of-02']

In [12]:
def preprocess_tfrecord(record):
  feature_desc={
      "review": tf.io.FixedLenFeature([], tf.string, b""),
      "label": tf.io.FixedLenFeature([], tf.int64, -1)
  }
  example = tf.io.parse_single_example(record, feature_desc)
  return example['review'], example['label']

def get_tensor_dataset_from_path(paths):
  dataset = tf.data.TFRecordDataset(paths, num_parallel_reads=10)
  dataset = dataset.map(preprocess_tfrecord)
  return dataset.batch(32).prefetch(1)

In [13]:
TRAIN_SET = get_tensor_dataset_from_path(TRAIN_PATHS)

In [14]:
VAL_SET = get_tensor_dataset_from_path(VALIDATE_PATHS)
TST_SET = get_tensor_dataset_from_path(TEST_PATHS)

In [15]:

def text_pre_process(X_batch, total_slots=50):
    # shape = tf.shape(X_batch) * tf.constant([1, 0]) + tf.constant([0, total_slots])
    shape = tf.shape(X_batch) * tf.constant([1, 0]) + tf.constant([0, total_slots])
    Z = tf.strings.substr(X_batch, 0, 300)
    Z = tf.strings.lower(Z)
    Z = tf.strings.regex_replace(Z, b"<br\\s*/?>", b" ")
    Z = tf.strings.regex_replace(Z, b"[^a-z]", b" ")
    Z = tf.strings.split(Z)
    return Z.to_tensor(shape=shape, default_value=b"<pad>")

In [16]:
from collections import Counter

def get_vol(X: tf.Tensor, most_common: int):
  Y = X.numpy()
  ct = Counter()
  for review in Y:
    for word in review:
      ct[word]+=1
  return [wd for wd, count in ct.most_common(most_common)]


In [17]:
def extract_vol(X: tf.Tensor, max_words_per_review, max_vol_size):
  return get_vol(text_pre_process(X, max_words_per_review), max_vol_size)

In [18]:
text_example = tf.constant([
                            "this is an example", "I feel very good"
])

In [19]:
extract_vol(text_example, 100, 1000)

[b'<pad>', b'this', b'is', b'an', b'example', b'i', b'feel', b'very', b'good']

Need to specify the data type for Layer. The default type is `float`

In [20]:
class TextVectorization(keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__(dtype=tf.string, **kwargs)
  
  def fit(self, examples):
    words = extract_vol(examples, 100, 1000)
    size = len(words)
    word_ids = tf.range(size, dtype=tf.int64)
    self.total_size = size + math.ceil(size*0.1)
    vocab_init = tf.lookup.KeyValueTensorInitializer(tf.constant(words), word_ids)
    self.table = tf.lookup.StaticVocabularyTable(vocab_init, math.ceil(size * 0.1))
  
  def call(self, data):
    result = text_pre_process(data, 100)
    return self.table.lookup(result)

  def get_total_size(self):
    return self.total_size

In [21]:
class BagOfWords(keras.layers.Layer):
  def __init__(self, depth, **kwargs):
    self.depth = depth
    super().__init__(**kwargs)
  
  def call(self, data):
    one_hot_code = tf.one_hot(data, self.depth)
    return tf.reduce_sum(one_hot_code, axis=1)

Get some samples 

In [23]:
sample = TRAIN_SET.take(50).map(lambda review, label: review)

In [24]:
concatenated_sample_reviews = np.concatenate(list(sample.as_numpy_iterator()), axis=0)

Create TextVencorization layer and bag of words layer

In [25]:
tv = TextVectorization(input_shape=[])
tv.fit(concatenated_sample_reviews)

In [26]:
bow = BagOfWords(tv.get_total_size())

In [27]:
tv(concatenated_sample_reviews)

<tf.Tensor: shape=(1600, 100), dtype=int64, numpy=
array([[  82,  215, 1077, ...,    0,    0,    0],
       [  51,   23,  260, ...,    0,    0,    0],
       [ 157,  745,    1, ...,    0,    0,    0],
       ...,
       [ 150, 1016,  657, ...,    0,    0,    0],
       [1079,  630,    9, ...,    0,    0,    0],
       [   1, 1059, 1038, ...,    0,    0,    0]])>

In [28]:
model = keras.models.Sequential([
                                 tv,
                                 bow,
                                 Dense(100, activation=keras.activations.relu),
                                 Dense(1, activation=keras.activations.sigmoid)
])

In [29]:
model.compile(optimizer="nadam", loss=keras.losses.binary_crossentropy, metrics=[keras.metrics.binary_accuracy])

In [30]:
model.fit(TRAIN_SET, epochs=5, validation_data=VAL_SET)

Epoch 1/5
782/782 [==============================] - 5s 7ms/step - loss: 0.5450 - binary_accuracy: 0.7166 - val_loss: 0.5155 - val_binary_accuracy: 0.7337
Epoch 2/5
782/782 [==============================] - 5s 6ms/step - loss: 0.4939 - binary_accuracy: 0.7542 - val_loss: 0.5121 - val_binary_accuracy: 0.7363
Epoch 3/5
782/782 [==============================] - 5s 6ms/step - loss: 0.4789 - binary_accuracy: 0.7632 - val_loss: 0.5118 - val_binary_accuracy: 0.7359
Epoch 4/5
782/782 [==============================] - 5s 6ms/step - loss: 0.4618 - binary_accuracy: 0.7738 - val_loss: 0.5120 - val_binary_accuracy: 0.7377
Epoch 5/5
782/782 [==============================] - 5s 6ms/step - loss: 0.4396 - binary_accuracy: 0.7876 - val_loss: 0.5160 - val_binary_accuracy: 0.7383
